<a href="https://colab.research.google.com/github/patshandofdoom/SECFindings/blob/main/SECScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a

Reference Data

[This is an edgar parser](https://github.com/rsljr/edgarParser). It can pull info from forms to show you what was written within




# Establish environment

In [1]:
#https://stackoverflow.com/questions/69025255/how-to-web-scraping-sec-edgar-10-k-dynamic-data
#removed the install of fake-useragent and import useragent from fake-useragent
!pip --quiet install reticker lxml google -U selenium
!pip install --quiet mysqlclient mysql-connector-python pandas
from bs4 import BeautifulSoup
import requests
import sys
import xml.etree.ElementTree as ET
import re
import pandas as pd

import os
import MySQLdb
import json
import mysql.connector
from mysql.connector import Error


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 29.4 MB/s eta 0:00:00


# Find New opportunities via the Benzinga live feed page


Open Benzinga web page, copy table and store info into databaasetable

---

Need to store items that have already been sent using the "Ticker" and "Date Announced" in a seperate file. Any new item should send a notification


In [ ]:
#all required packages
from datetime import date
from dateutil.relativedelta import relativedelta

#reticker makes it easier to regex tickers from a string
import reticker
extractor = reticker.TickerExtractor()

#set up the database credentials for each of the functions
with open('/content/drive/MyDrive/Colab Notebooks/SEC Scraping/Scraping.txt','r') as readfile:
  credentials = readfile.read()
  credentials = json.loads(credentials)

dbhost = credentials["HOST"]
dbuser = credentials["USERNAME"]
dbpasswd= credentials["PASSWORD"]
dbname = credentials["DATABASE"]

##########################################################################################
#if day and month have <2 characters, add a 0
def adddigit(characters):
  if (len(characters) < 2):
    return( "0"+characters)
  else:
    return(characters)

###############################################################################
  #This section builds the HTML code using today's date onward and fetches the response

def URLsoupFind():
  #get todays date
  today = date.today()
  startyear = str(today.year)
  startmonth = adddigit(str(today.month))
  startday = adddigit(str(today.day))

  #get date 2 months out
  enddate = date.today() + relativedelta(months=+2)
  endyear = str(enddate.year)
  endmonth = adddigit(str(enddate.month))
  endday = adddigit(str(enddate.day))


  base_url = f"https://www.benzinga.com/calendars/stock-splits?date_from={startyear}-{startmonth}-{startday}&date_sort=ex&date_to={endyear}-{endmonth}-{endday}"
  theseheaders = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}
  print(base_url)
  benzingaresponse = requests.get(base_url,headers=theseheaders)

  benzingasoup = BeautifulSoup(benzingaresponse.text)
  return benzingasoup

###############################################################################
#this segment checks live table on the benzinga website


###############################################################################
#this segment checks the live feed on the benzinga website
def shortfeed(benzingasoup):
  mydivs = benzingasoup.findAll('div',class_="mb-4")
  #pull out all of the 'a' divisions
  allupdates = mydivs[0].findAll('a')
  finalarray = []

  for updates in allupdates:
    #find all the spans in 'a' and make them into an array
    updatetitlespans = updates.findAll('span')

    #Sometimes the 'a' segments will have 3 spans, others will have more.
    #If it has 3 spans, we can grab the info from span 3 for the
    if(len(updatetitlespans)>3):
      stockticker = re.findall("\:(.*?)\)",updatetitlespans[2].text)
      #ran into an issue where one of the parentheses included 2 tickers. so I added a clause here to split those adn take the first.
      #Need to make sure the stockticker has length, then split it on the comma and take the first
      if len(stockticker) >0:
        if "," in stockticker[0]:
            stockticker1 = re.split(",",stockticker[0])
            stockticker = [stockticker1[0]]

      instancetext = updatetitlespans[2].text

    #Otherwise, we grab the story text from span 2 and see if span 3 has any info about the ticker
    else:
      stockticker = re.findall("\:(.*?)\)",updatetitlespans[2].text)
      #ran into an issue where one of the parentheses included 2 tickers. so I added a clause here to split those adn take the first.
      #Need to make sure the stockticker has length, then split it on the comma and take the first
      if len(stockticker) >0:
        if "," in stockticker[0]:
            stockticker1 = re.split(",",stockticker[0])
            print(stockticker1)
            stockticker = [stockticker1[0]]

      instancetext = updatetitlespans[1].text
      instancetext = instancetext.replace('"','')
    #If we cant find the ticker, try using extractor
    if len(stockticker) == 0:
      stockticker = extractor.extract(updatetitlespans[2].text)
    #If it still cant be found, make it N/A
    if len(stockticker) == 0:
      stockticker = "N/A"
    else:
      #finally, remove all space caharcters from the ticker
      stockticker = stockticker[0].replace(u'\xa0', '')
      stockticker = stockticker.replace(' ', '')
    #The split title will be in the second span
    instancetitle = updatetitlespans[1].text
    #replace all quotation marks in the title adn text with apostrophes
    instancetitle = instancetitle.replace('"',"'")
    instancetext = instancetext.replace('"',"'")

    instancedict = {'ticker':stockticker,
                 'title':instancetitle,
                 'articletext':instancetext}
    finalarray.append(instancedict)
  return finalarray

###############################################################################
#this function creates the general google search that I usually do by hand for each of these tickers
#it returns the HTML link of the first result of that search
def searchresults(ticker):
  try:
      from googlesearch import search
  except ImportError:
      print("No module named 'google' found")

  # to search
  query = 'reverse split fractional "'+ticker+'" site:yahoo.com'

  for j in search(query, tld="co.in", num=10, stop=10, pause=2):
      return(j)

###############################################################################
#next it need to keep a database of already sent items, then send the items to the discord bot
#pass in a dictionary of columns to search and values to search for then pass out the result. If it is an array with no length, you are good. otherwise there is a matching entry
def check_for_entries(whichtable):
  #initiate Connection with the database
  connection = mysql.connector.connect(host = dbhost, user = dbuser, passwd= dbpasswd, db = dbname, autocommit = True, )
  #Try connecting and working with the database
  try:
    if connection.is_connected():
        #Create a query which will return several rows of data that we can search
        mySql_Create_Table_Query = "SELECT max(id_number), title FROM "+whichtable+" GROUP BY id_number LIMIT 100"
        cursor = connection.cursor(buffered=True)
        cursor.execute(mySql_Create_Table_Query)
        result = cursor.fetchall()

  except Error as e:
      print("Error while connecting to MySQL", e)
  finally:
      if connection.is_connected():
          cursor.close()
          connection.close()
          print("Search connection is closed")
      return result

###############################################################################
#This function takes in a dictionary of the table values and compares it to the lates database entries
def checkformatch(eachdictionary,latestdatabaseentries):
  found = False
  i=0

  while found==False and  i<len(latestdatabaseentries):

    if(latestdatabaseentries[i][1]==eachdictionary["title"]):
      found = True
    i+=1
  return found

###############################################################################

#pass in the dictionary that we are going to write to and which table we are writing to
def write_entries(whatdictionary,whichtable):
  #initiate Connection with the database
  connection = mysql.connector.connect(host = dbhost, user = dbuser, passwd= dbpasswd, db = dbname, autocommit = True, )
  #Try connecting and working with the database
  try:
    if connection.is_connected():
        #build the query to search in SQL. This adds the search
        databaseheaders=""
        databasevalues=""
        counter = 0
        keys = whatdictionary.keys()
        for each in keys:
          databaseheaders+= each
          databasevalues+='"'+whatdictionary[each]+'"'
          counter+=1
          if counter != len(keys):
            databaseheaders+=","
            databasevalues+=","

        #Submit the addition
        mySql_Create_Table_Query = "insert into "+whichtable+"("+databaseheaders+") VALUES("+databasevalues+")"
        print(mySql_Create_Table_Query)
        cursor = connection.cursor(buffered=True)
        cursor.execute(mySql_Create_Table_Query)

  except Error as e:
      print("Error while connecting to MySQL", e)
  finally:
      if connection.is_connected():
          cursor.close()
          connection.close()
          print("Write connection is closed")

###############################################################################
# Function that sends a message to the Discord channel
def send_message(ticker, title, text, url):
    WEBHOOK_URL = 'https://discord.com/api/webhooks/1102332754965831822/DOQYwrS8TAE8Es1EaS44A-z58aANgkwuXXTqcIslCS0IKTKm_l3Mt1Dr_xnlnpVHyKHd'
    message = f'TICKER: {ticker}\n TITLE: {title}\n DETAILS: {text}\n URL: {url}'
    data = {'content': message}
    requests.post(WEBHOOK_URL, json=data)

###############################################################################
###############################################################################
###############################################################################
#Begin Process.
#First grab the live table from benzinga, then grab the full table from benzinga
#store each of the items from each table into their respective database tables

#get the soup, get the live table and parse it.
originalSoup = URLsoupFind()
livetablefinalarray = shortfeed(originalSoup)
latestdatabaseentries = check_for_entries('benzingaliveupdates')

#for each of the values in the benzinga live table, check if its in the database already
for eachdictionary in livetablefinalarray:
  print(eachdictionary)
  #check for a match in database data
  matchfound = checkformatch(eachdictionary,latestdatabaseentries)

  #If no match is found, do the google search for me add a url variable
  if matchfound ==False:
    #If it didnt find the ticker, just add a dummy variable for the URL
    if eachdictionary['ticker'] == "N/A":
      eachdictionary['url'] =""
    else:
      #Otherwise do the google search and log the first result
      ticker = eachdictionary['ticker']
      searchURL = searchresults(ticker)
      eachdictionary['url'] =searchURL
    #Add the roundup value of ? since we do not know yet if it will round up
    eachdictionary['roundup'] = "?"
    #then write it to the database
    write_entries(eachdictionary,"benzingaliveupdates")
    #finally send a message
    print(eachdictionary)
    send_message(ticker, eachdictionary['title'], eachdictionary['articletext'],searchURL)


  else:
    print("This entry is already in the database")

# Find new opportunities in the main benzinga table

This is an attempt to access the main benzinga table. unfortunately its loaded via javascript so I will need to use selenium instead of just a get function.

In [ ]:
###https://saturncloud.io/blog/how-to-use-selenium-webdriver-in-google-colab/

# this section will load the main table from the
#all required packages
from datetime import date
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import re
#reticker makes it easier to regex tickers from a string
import reticker
extractor = reticker.TickerExtractor()

#if day and month have <2 characters, add a 0
def adddigit(characters):
  if (len(characters) < 2):
    return( "0"+characters)
  else:
    return(characters)

###############################################################################
#This section builds the HTML code using today's date onward and fetches the response

def URLsoupFind():
  #get todays date
  today = date.today()
  startyear = str(today.year)
  startmonth = adddigit(str(today.month))
  startday = adddigit(str(today.day))

  #get date 2 months out
  enddate = date.today() + relativedelta(months=+2)
  endyear = str(enddate.year)
  endmonth = adddigit(str(enddate.month))
  endday = adddigit(str(enddate.day))

  base_url = f"https://www.benzinga.com/calendars/stock-splits?date_from={startyear}-{startmonth}-{startday}&date_sort=ex&date_to={endyear}-{endmonth}-{endday}"
  #///////////////////////////////////
  browser = webdriver.chrome.webdriver.WebDriver
  browser.get(base_url)
  html = browser.page_source
  soup = BeautifulSoup(html, 'lxml')
  a = soup.findAll('tbody')
  #///////////////////////////////////

  #theseheaders = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}
  #print(base_url)
  #benzingaresponse = requests.get(base_url,headers=theseheaders)

  benzingasoup = BeautifulSoup(a.text)
  return benzingasoup

########################################################
benzingasoup = URLsoupFind()
print(benzingasoup)
tables = benzingasoup.findAll('tr')
print(tables)
#pull out all of the 'a' divisions
tablerows = tables[0].findAll('tr')
print(tablerows)
finalarray = []

for row in tablerows:
  #find all the spans in 'a' and make them into an array
  tabledata = allupdates.findAll('td')
  for datum in tabledata:
    print(datum)



TypeError: ignored

# Finding the files that we need to search on SEC RSS Feed

In [ ]:
#establish environment
import time
import pandas as pd
import requests
import xml.etree.ElementTree as ET

#Log the url we need for the RSS feed of 8K documents from he SEC
thisurl = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcurrent&CIK=&type=8&company=&dateb=&owner=include&start=0&count=40&output=atom"
#Create the headers to use on each of the html requests
theseheaders = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}
#loads the CIK data as a dataframe so that we dont need to load it a million times
cikdf = pd.read_csv("https://www.sec.gov/include/ticker.txt", delimiter='\t', header=None)
cikdf = cikdf.set_index(1)

##########################################################################################
#https://punit-arani.medium.com/get-the-cik-number-for-a-stock-using-python-2eaa494fbd91
#This function will take a CIK number and translate it into a ticker.
def gettickerfromCIK(ciknumber):
    #return ticker if it exists
    try:
      ticker = cikdf.loc[int(ciknumber), 0]
      if isinstance(ticker,pd.Series):
        ticker = ticker.iloc[0]
      print(ticker)
      return ticker

    #return None there's no ticker for the CIK
    except:
      return None

##########################################################################################
#This function will take the SEC 8k filing URL and search for the top 40 newest 8K filings, returning a matrix of dictionaries
def retrieve8klist():
  edgar_resp = requests.get(thisurl,headers=theseheaders)
  thissoup = edgar_resp.text
  #important to have a list of filings
  formlist = []
  root = ET.fromstring(thissoup)
  #for all child elements check if it's an entry
  for child in root:
    if child.tag =="{http://www.w3.org/2005/Atom}entry":
      #Then grab the title and href from each entry.
      for nestedchild in child:
        if nestedchild.tag == "{http://www.w3.org/2005/Atom}title":
          fulltitle = nestedchild.text
          formtype = fulltitle.split(" - ")
          formtype = formtype[0]
        if nestedchild.tag == "{http://www.w3.org/2005/Atom}link":
          formlink = nestedchild.attrib['href']
      childdict = {'formtype':formtype,'fulltitle':fulltitle, 'link':formlink}
      formlist.append(childdict)


  return(formlist)

##########################################################################################

#this function will get the direct link to the 8K form
def getformlinkfromfilinglink(filinglink):
  #open the link and get the soup
  edgar_resp = requests.get(filinglink,headers=theseheaders)
  filingsoup = BeautifulSoup(edgar_resp.text)

  #this segment of code finds the CIK number and grabs the ticker
  companyinfo = filingsoup.find('span', class_='companyName')
  ciknumber = companyinfo.find('a').text
  ciknumber = re.split(" \([^)]*\)",ciknumber)
  ciknumber = ciknumber[0]
  ticker = gettickerfromCIK(ciknumber)

  #from the soup, we get the table
  tableentries = filingsoup.findAll('table',class_='tableFile')
  linktoform = ""
  #search through the rows for all data
  for eachrow in tableentries[0].findAll('tr'):
    celldata = eachrow.findAll('td')
    #As long as this row has tr labels and the type is an 8-k form, get the link to the 8-k
    if len(celldata)>0:
      if celldata[3].text=='8-K':
        for link in celldata[2].findAll('a',href=True):
          link= link['href'].replace("ix?doc=","")
          linktoform = "https://sec.gov"+link

  return({'cik':ciknumber,'ticker':ticker,'formlink':linktoform})

##########################################################################################
#This takes the list from the Retrieve8k list and opens each link, parsing the internal data and returning the count
def parse8klist(entry):
  #this block here to open the filing page and get the right link from the table
  if(entry['formtype']=='8-K'):
    print(entry['link'])
    #This function creates a dictionary which we can pass up to the database after we get all info
    filinginformation = getformlinkfromfilinglink(entry['link'])
    print(filinginformation['formlink'])

    #find the filing URL in the webpage
    edgar_resp = requests.get(filinginformation['formlink'],headers=theseheaders)

    #Access the Filing
    formsoup = edgar_resp.text
    formsoup = formsoup.lower()

    #find all instances of word reverse and word merger
    revCount = re.findall(r'reverse',formsoup,re.MULTILINE)
    mergercount = re.findall(r'merger',formsoup,re.MULTILINE)
    #reverse count - count the number of times Reverse is in the text
    print(len(revCount))

    #merger count
    print(len(mergercount))
    time.sleep(1)
    return([])

##########################################################################################
##########################################################################################
##########################################################################################
#Retrieve list of the 40 latest 8k forms filed
rssof8kforms = retrieve8klist()
#need a function here to get the

#for each of those forms,
for eachform in rssof8kforms:
  parse8klist(eachform)


ticker = gettickerfromCIK()


https://www.sec.gov/Archives/edgar/data/1269371/000106823823000157/0001068238-23-000157-index.htm
0001269371
https://sec.gov/Archives/edgar/data/1269371/000106823823000157/sntlt2003-5_8k.htm
0
0
https://www.sec.gov/Archives/edgar/data/1264157/000106823823000156/0001068238-23-000156-index.htm
0001264157
https://sec.gov/Archives/edgar/data/1264157/000106823823000156/sntlt2003-4_8k.htm
0
0


KeyboardInterrupt: ignored

This code block is a snippet which will grab data from a filing URL and parse it for how many mentions it contains to reverse or merger.

The code in this block is here to test expanding the parse8klist() function above

In [ ]:
#headers = ua.random
headers = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}

# Access page
cik = '200406'
type = '10-K'
dateb = '20210704'

# Obtain HTML for search page
#base_url = f"https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={cik}&type={type}&dateb={dateb}"
base_url = 'ix?doc=/Archives/edgar/data/101984/000010198423000034/ueic-20230605.htm'

#have to split off the ix?doc bullcrap, get righ to the HTM link
link= base_url.split("ix?doc=")
base_url = 'https://www.sec.gov'+link[1]
edgar_resp = requests.get(base_url,headers=headers)

thissoup = edgar_resp.text
thissoup = thissoup.lower()

#reverse count - count the number of times Reverse is in the text
print(len(re.findall(r'reverse',thissoup,re.MULTILINE)))

#merger count
print(len(re.findall(r'universal',thissoup,re.MULTILINE)))


0
4


This code below is to try and find the CIP list number in order to grab the ticker

In [ ]:
import pandas as pd

#loads the CIK data as a dataframe so that we dont need to load it a million times
cikdf = pd.read_csv("https://www.sec.gov/include/ticker.txt", delimiter='\t', header=None)
cikdf = cikdf.set_index(1)
def gettickerfromCIK(ciknumber):
    #return ticker if it exists
    try:
      ticker = cikdf.loc[int(ciknumber), 0]
      if isinstance(ticker,pd.Series):
        ticker = ticker.iloc[0]
      return ticker
    #return None there's no ticker for the CIK
    except:
      return None

###############################################################################
headers = {"User-agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36"}
url = "https://www.sec.gov/Archives/edgar/data/1042776/000119312523193273/0001193125-23-193273-index.htm"
edgar_resp = requests.get(url,headers=headers)
thissoup = BeautifulSoup(edgar_resp.text)

companyinfo = thissoup.find('span', class_='companyName')
ciknumber = companyinfo.find('a').text

ciknumber = re.split(" \([^)]*\)",ciknumber)
ciknumber = ciknumber[0]

print(ciknumber)

ticker = gettickerfromCIK(ciknumber)

print(ticker)

0001042776
pdm


# Access Database


This is a segment will attach to the SQL databsae and read data

In [2]:
#Open credentials file and load the JSON variables inside
with open('/content/drive/MyDrive/Colab Notebooks/SEC Scraping/Scraping.txt','r') as readfile:
  credentials = readfile.read()
  credentials = json.loads(credentials)

#initiate Connection with the database
connection = mysql.connector.connect(
  host = credentials["HOST"],
  user = credentials["USERNAME"],
  passwd= credentials["PASSWORD"],
  db = credentials["DATABASE"],
  autocommit = True,
)

#Try connecting and working with the database
try:
  if connection.is_connected():
      db_Info = connection.get_server_info()
      print("Connected to MySQL Server version ", db_Info)
      cursor = connection.cursor(buffered=True)
      cursor.execute("select database();")
      record = cursor.fetchone()
      print("You're connected to database: ", record)

      ##this section is where the query code will go
      mySql_Create_Table_Query = "SELECT * FROM benzingaliveupdates"

      #check if returned data contains the file already already



      #""CREATE TABLE Laptop (Id int(11) NOT NULL, Name varchar(250) NOT NULL, Price float NOT NULL, Purchase_date Date NOT NULL, PRIMARY KEY (Id)) ""
      cursor = connection.cursor(buffered=True)
      cursor.execute(mySql_Create_Table_Query)
      result = cursor.fetchall()
      for row in result:
        print(row)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")


Connected to MySQL Server version  8.0.23-PlanetScale
You're connected to database:  ('stock_live_updates',)
(1, 'BTCY', 'Biotricity Announces 1-For-6 Reverse Stock Split', '\xa0Biotricity Holdings, Inc. ("Biotricity" or the "Company") (NASDAQ:BTCY), a medical diagnostic and consumer healthcare technology company, today it will effect a reverse stock split of its common stock', None, None, None, None)
(2, 'AHI', 'Advanced Health Intelligence Announced Implementation of ADS Ratio Change With Effect Of 1:4 Reverse ADS Split', '- Advanced Health Intelligence Ltd (NASDAQ:AHI) (“Advanced Health Intelligence”, “AHI” or the “Company”) announces today that its planned ratio change of the Company’s American', None, None, None, None)
(5, 'N/A', 'Accelerate Diagnostics Announces 1-For-10 Reverse Stock Split', 'Accelerate Diagnostics Announces 1-For-10 Reverse Stock Split', None, None, '', '?')
(6, 'AAON', 'AAON Adopts 3-For-2 Stock Split', 'AAON Inc\xa0(NASDAQ: AAON) said the Board of Directors\x

This segment below will delete values in the databsae

In [3]:
#https://app.planetscale.com/prichardson0874/stock_live_updates
#https://planetscale.com/docs/tutorials/planetscale-quick-start-guide
#https://pynative.com/python-mysql-database-connection/
#https://stackoverflow.com/questions/54571009/how-to-hide-secret-keys-in-google-colaboratory-from-users-having-the-sharing-lin

#Open credentials file and load the JSON variables inside
with open('/content/drive/MyDrive/Colab Notebooks/SEC Scraping/Scraping.txt','r') as readfile:
  credentials = readfile.read()
  credentials = json.loads(credentials)

#initiate Connection with the database
connection = mysql.connector.connect(
  host = credentials["HOST"],
  user = credentials["USERNAME"],
  passwd= credentials["PASSWORD"],
  db = credentials["DATABASE"],
  autocommit = True,
)

#Try connecting and working with the database
try:
  if connection.is_connected():
      db_Info = connection.get_server_info()
      print("Connected to MySQL Server version ", db_Info)
      cursor = connection.cursor()
      cursor.execute("select database();")
      record = cursor.fetchone()
      print("You're connected to database: ", record)

      ##this section is where the query code will go
      mySql_Create_Table_Query = """delete FROM benzingaliveupdates WHERE id_number=58"""

      #""CREATE TABLE Laptop (Id int(11) NOT NULL, Name varchar(250) NOT NULL, Price float NOT NULL, Purchase_date Date NOT NULL, PRIMARY KEY (Id)) ""
      cursor = connection.cursor()
      result = cursor.execute(mySql_Create_Table_Query)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.23-PlanetScale
You're connected to database:  ('stock_live_updates',)
MySQL connection is closed


Need to turn the database call and response into a function passing in the database table name and the SQL function since we will be connecting a few times.  we need a section of code which

# Access the Sheet for URLs
This segment will access the google sheet that I made in order to check each of the URL's for me. This should save a significant amount of time. I am still frustrated that I couldnt get this to work in google sheets natively, but I guess Python is just better in this way.

Importing data from google sheets
https://docs.gspread.org/en/v5.7.0/

The current work-around would be to use this segment first, then run the analysis segment with a sotred data table

In [ ]:
#https://docs.gspread.org/en/v5.7.2/user-guide.html#getting-a-cell-value
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

#Open the stockwatch sheet
sh = gc.open_by_key('1gL6YuJdplpqQd3H4pHWZGqK6W7YDMkWkGmg3OswNcao')

#Open the Rev-split worksheet
worksheet = sh.worksheet('Rev-Spl Find')

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)

Access and Write to Database

# Original Code

The code below is the original version of the fetch code that I was able to retrieve results



In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

#sh = gc.create('A new spreadsheet')

# Open our new sheet and add some data.
worksheet = gc.open('Stock Watch').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)
# Go to https://sheets.google.com to see your new spreadsheet.